# Versatility of CUAD models

The aim of this notebooks is to create more questions for the CUAD dataset to increase the size of the dataset and potentially increase the versatility. 

It also adds a set of questions only for a seperate test set with unseen questions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_desc = pd.read_csv('../data/category_descriptions.csv')
df_desc.head(50)

,Category (incl. context and answer),Description,Answer Format,Group
0,Category: Document Name,Description: The name of the contract,Answer Format: Contract Name,Group: -
1,Category: Parties,Description: The two or more parties who signe...,Answer Format: Entity or individual names,Group: -
2,Category: Agreement Date,Description: The date of the contract,Answer Format: Date (mm/dd/yyyy),Group: 1
3,Category: Effective Date,Description: The date when the contract is eff...,Answer Format: Date (mm/dd/yyyy),Group: 1
4,Category: Expiration Date,Description: On what date will the contract's ...,Answer Format: Date (mm/dd/yyyy) / Perpetual,Group: 1
5,Category: Renewal Term,Description: What is the renewal term after th...,Answer Format: [Successive] number of years/mo...,Group: 1
6,Category: Notice Period to Terminate Renewal,Description: What is the notice period require...,Answer Format: Number of days/months/year(s),Group: 1
7,Category: Governing Law,Description: Which state/country's law governs...,Answer Format: Name of a US State / non-US Pro...,Group: -
8,Category: Most Favored Nation,Description: Is there a clause that if a third...,Answer Format: Yes/No,Group: -
9,Category: Non-Compete,Description: Is there a restriction on the abi...,Answer Format: Yes/No,Group: 2


The default formulation of question are 

"Highlight the parts (if any) of this contract related to {<span style="color:red">Category</span>} that should be reviewed by a lawyer.
Details: {<span style="color:red">Description</span>(minus the "Description:")}
Thus we aim to make the questions in a way where we can generate the new questions automatically.

In [3]:
q = set()
for x in df_desc.apply(lambda x: f"Highlight the parts (if any) of this contract related to \"{x['Category (incl. context and answer)'].split(': ')[-1]}\" that should be reviewed by a lawyer. Details: {x['Description'].split('Description: ')[-1]}", axis=1).values:
    #x=x.replace("Notice Period to Terminate Renewal", "Notice Period To Terminate Renewal")
    #x=x.replace("No Solicitation Of Customers", "No-Solicit Of Customers")
    q.add(x)

This is exactly the structure from the dataset. We will now create addition this this in various degree of difficulty. The goals is that the models are more susceptible to "normal" human wording. 

E.G. that is someone can ask: What is the document name? Who are the parties in this contract. Furthermore, just rearrangements of the words etc. should work

## A new test set with different wordings
We will do three different degrees of difficulty

1. Remove casing
2. Rearrangement of the question and answer
3. Different wording buy close to the original
4. Completely different natural wording

All experiments will reformulate on the test.json dataset into a new version.

In [4]:
import json
# Open the json file and load the data
def load_json(file):
    with open(file) as f:
        data = json.load(f)
    return data

def dump_json(file, data):
    with open(file, 'w') as f:
        json.dump(data, f)

In [5]:
def change_question(file,question_map, drop_question_not_found=False, return_unique_q=False):  
    dropped=0
    total=0      
    data = load_json(file)
    unique_q_used = {}
    for contract in data['data']:
        for paragraph in contract['paragraphs']:
            for qa in paragraph['qas']:
                total+=1
                if qa['question'].lower() not in question_map:
                    if drop_question_not_found:
                        dropped+=1
                        continue
                    # Raise exception
                    raise Exception(f"{qa['question']} not in question_map")
                unique_q_used[qa['question']]=question_map[qa['question'].lower()]
                qa['question'] = question_map[qa['question'].lower()]
    if drop_question_not_found:
        print(f"Dropped {dropped} questions not found in question_map out of {total} questions")
    
    if return_unique_q:
        return data, unique_q_used
    return data


**1. To lower**

This script just changes the low 

In [6]:
from transformers import AutoTokenizer
tk=AutoTokenizer.from_pretrained('roberta-base')

In [7]:
data = load_json('../data/test.json')
question_map = {k.lower(): k.lower() for k in q}
data_lower = change_question('../data/test.json', question_map)

In [12]:
# Calculate the jaccard distance between the two lists of integers
def jaccard_distance(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = len(list(set(list1).union(list2)))
    return 1 - (intersection / union)


In [25]:
# Calculate the jaccard distance between the two tokenized questions
jaccard_data = []
for k,v in question_map.items():
    l1 = tk.encode(k)
    l2 = tk.encode(v)
    jaccard_data.append((k,v,jaccard_distance(l1,l2)))

    
np.mean([x[2] for x in jaccard_data])

0.0

In [26]:
dump_json('../data/test_lower.json', data_lower)

**2. Rearrangement of the questions and answers**

This first script simply reverses the order of question and details formulation

In [27]:
q_map_reverse_order = {}
for row in df_desc.iterrows():
    original = f"Highlight the parts (if any) of this contract related to \"{row[1]['Category (incl. context and answer)'].split(': ')[-1]}\" that should be reviewed by a lawyer. Details: {row[1]['Description'].split('Description: ')[-1]}"
    reversed_ = f"Details: {row[1]['Description'].split('Description: ')[-1]}. Highlight the parts (if any) of this contract related to \"{row[1]['Category (incl. context and answer)'].split(': ')[-1]}\" that should be reviewed by a lawyer."
    q_map_reverse_order[original.lower()] = reversed_

# Create dataset with new map
data_lower_reversed = change_question('../data/test.json', q_map_reverse_order)
dump_json('../data/test_reversed.json', data_lower_reversed)

**3. Different wording buy close to the original**

This script changes the wording of the questions, but it's still very close to the original

In [28]:
#Manual mapping
q_map_different_wording = {
    "highlight the parts (if any) of this contract related to \"document name\" that should be reviewed by a lawyer. details: the name of the contract": 
"Show the sections of this contract (if there are any) related to the \"Document Name\" . This is the name of the contract.",

    "highlight the parts (if any) of this contract related to \"parties\" that should be reviewed by a lawyer. details: the two or more parties who signed the contract": 
"Show the sections of this contract (if there are any) related to the \"Parties\" of the agreement. This is the entities/parties who signed the contract",

    "highlight the parts (if any) of this contract related to \"agreement date\" that should be reviewed by a lawyer. details: the date of the contract": 
"Show the sections of this contract (if there are any) related to the \"Agreement Date\". This is the date of the contract",

    "highlight the parts (if any) of this contract related to \"effective date\" that should be reviewed by a lawyer. details: the date when the contract is effective\xa0": 
"Show the sections of this contract (if there are any) related to the \"Effective Date\". This is the date when the contract is effective",

    "highlight the parts (if any) of this contract related to \"expiration date\" that should be reviewed by a lawyer. details: on what date will the contract's initial term expire?": 
"Show the sections of this contract (if there are any) related to the \"Expiration Date\". This is the date when the contract's initial term expires",

    "highlight the parts (if any) of this contract related to \"renewal term\" that should be reviewed by a lawyer. details: what is the renewal term after the initial term expires? this includes automatic extensions and unilateral extensions with prior notice.": 
"Show the sections of this contract (if there are any) related to the \"Renewal Term\". This is the renewal term after the initial term expires and includes automatic extensions and unilateral extensions with prior notice.",

    "highlight the parts (if any) of this contract related to \"notice period to terminate renewal\" that should be reviewed by a lawyer. details: what is the notice period required to terminate renewal?": 
"Show the sections of this contract (if there are any) related to the \"Notice Period To Terminate Renewal\". This is the notice period required to terminate renewal",

    "highlight the parts (if any) of this contract related to \"governing law\" that should be reviewed by a lawyer. details: which state/country's law governs the interpretation of the contract?": 
"Show the sections of this contract (if there are any) related to the \"Governing Law\". This which state or country's law governs the interpretation of the contract",

    "highlight the parts (if any) of this contract related to \"most favored nation\" that should be reviewed by a lawyer. details: is there a clause that if a third party gets better terms on the licensing or sale of technology/goods/services described in the contract, the buyer of such technology/goods/services under the contract shall be entitled to those better terms?": 
"Show the sections of this contract (if there are any) related to the \"Most Favored Nation\". This is a clause that if a third party gets better terms on the licensing or sale of technology/goods/services described in the contract, the buyer of such technology/goods/services under the contract shall be entitled to those better terms",

    "highlight the parts (if any) of this contract related to \"non-compete\" that should be reviewed by a lawyer. details: is there a restriction on the ability of a party to compete with the counterparty or operate in a certain geography or business or technology sector?\u00a0": 
"Show the sections of this contract (if there are any) related to the \"Non-Compete\". This is a restriction on the ability of a party to compete with the counterparty or operate in a certain business or technology or geography sector",

    "highlight the parts (if any) of this contract related to \"exclusivity\" that should be reviewed by a lawyer. details: is there an exclusive dealing\u00a0 commitment with the counterparty? this includes a commitment to procure all \u201crequirements\u201d from one party of certain technology, goods, or services or a prohibition on licensing or selling technology, goods or services to third parties, or a prohibition on\u00a0 collaborating or working with other parties), whether during the contract or\u00a0 after the contract ends (or both).": 
"Show the sections of this contract (if there are any) related to the \"Exclusivity\". This is an exclusive dealing commitment with the counterparty and includes a commitment to procure all requirements from one party of certain services, technology, or goods or a prohibition on licensing or selling technology, goods or services to third parties, or a prohibition on working or collaborating with other parties), whether during the contract, after the contract ends, or both",

    "highlight the parts (if any) of this contract related to \"no-solicit of customers\" that should be reviewed by a lawyer. details: is a party restricted from contracting or soliciting customers or partners of the counterparty, whether during the contract or after the contract ends (or both)?": 
"Details: Is a party restricted from contracting or soliciting customers or partners of the counterparty, whether during the contract or after the contract ends (or both)?. Highlight the parts (if any) of this contract related to \"No-Solicit of Customers\" that should be reviewed by a lawyer.",

    "highlight the parts (if any) of this contract related to \"competitive restriction exception\" that should be reviewed by a lawyer. details: this category includes the exceptions or carveouts to non-compete, exclusivity and no-solicit of customers above.": 
"Show the sections of this contract (if there are any) related to the \"Competitive Restriction Exception\". This is an exception or carveouts to the non-compete, exclusivity, and no-solicit of customers clauses",

    "highlight the parts (if any) of this contract related to \"no-solicit of employees\" that should be reviewed by a lawyer. details: is there a restriction on a party\u2019s soliciting or hiring employees and/or contractors from the\u00a0 counterparty, whether during the contract or after the contract ends (or both)?": 
"Show the sections of this contract (if there are any) related to the \"No-Solicit of Employees\". This is a restriction on a party's soliciting or hiring employees and/or contractors from the counterparty",

    "highlight the parts (if any) of this contract related to \"non-disparagement\" that should be reviewed by a lawyer. details: is there a requirement on a party not to disparage the counterparty?": 
"Show the sections of this contract (if there are any) related to the \"Non-Disparagement\". This is a requirement on a party not to disparage the counterparty",

    "highlight the parts (if any) of this contract related to \"termination for convenience\" that should be reviewed by a lawyer. details: can a party terminate this\u00a0 contract without cause (solely by giving a notice and allowing a waiting\u00a0 period to expire)?": 
"Show the sections of this contract (if there are any) related to the \"Termination for Convenience\". Can party terminate this contract without cause",

    "highlight the parts (if any) of this contract related to \"rofr/rofo/rofn\" that should be reviewed by a lawyer. details: is there a clause granting one party a right of first refusal, right of first offer or right of first negotiation to purchase, license, market, or distribute equity interest, technology, assets, products or services?": 
"Show the sections of this contract (if there are any) related to the \"Right of First Refusal, Right of First Offer, Right of First Negotiation\". This is a clause granting one party a right of first refusal, right of first offer or right of first negotiation to purchase, license, market, or distribute equity interest, technology, assets, products or services",

    "highlight the parts (if any) of this contract related to \"change of control\" that should be reviewed by a lawyer. details: does one party have the right to terminate or is consent or notice required of the counterparty if such party undergoes a change of control, such as a merger, stock sale, transfer of all or substantially all of its assets or business, or assignment by operation of law?": 
"Show the sections of this contract (if there are any) related to the \"Change of Control\". Does one party have the right to terminate or is consent or notice required of the counterparty if such party undergoes a change of control, such as a stock sale, merger, transfer of all or substantially all of its assets or business, or assignment by operation of law?",

    "highlight the parts (if any) of this contract related to \"anti-assignment\" that should be reviewed by a lawyer. details: is consent or notice required of a party if the contract is assigned to a third party?": 
"Show the sections of this contract (if there are any) related to the \"Anti-Assignment\". Is consent or notice required of a party if the contract is assigned to a third party?",

    "highlight the parts (if any) of this contract related to \"revenue/profit sharing\" that should be reviewed by a lawyer. details: is one party required to share revenue or profit with the counterparty for any technology, goods, or\u00a0services?": 
"Show the sections of this contract (if there are any) related to the \"Revenue/Profit Sharing\". This is a clause related to if one party is required to share revenue or profit with the counterparty for any services, technology, or goods?",

    "highlight the parts (if any) of this contract related to \"price restrictions\" that should be reviewed by a lawyer. details: is there a restriction on the\u00a0 ability of a party to raise or reduce prices of technology, goods, or\u00a0 services provided?": 
"Show the sections of this contract (if there are any) related to the \"Price Restrictions\". This is a restriction on the ability of a party to raise or reduce prices of technology, goods, or services provided",

    "highlight the parts (if any) of this contract related to \"minimum commitment\" that should be reviewed by a lawyer. details: is there a minimum order size or minimum amount or units per-time period that one party must buy from the counterparty under the contract?": 
"Show the sections of this contract (if there are any) related to the \"Minimum Commitment\". This is a minimum minimum amount, order size or units per-time period that one party has to buy from the counterparty under the contract",

    "highlight the parts (if any) of this contract related to \"volume restriction\" that should be reviewed by a lawyer. details: is there a fee increase or consent requirement, etc. if one party\u2019s use of the product/services exceeds certain threshold?": 
"Show the sections of this contract (if there are any) related to the \"Volume Restriction\". This is a fee consent or increase requirement, etc. if one party's use of the product/services exceeds certain threshold",

    "highlight the parts (if any) of this contract related to \"ip ownership assignment\" that should be reviewed by a lawyer. details: does intellectual property created\u00a0 by one party become the property of the counterparty, either per the terms of the contract or upon the occurrence of certain events?": 
"Show the sections of this contract (if there are any) related to the \"IP Ownership Assignment\". Does intellectual property created by one party become the property of the counterparty, either upon the occurrence of certain events or per the terms of the contract?",

    "highlight the parts (if any) of this contract related to \"joint ip ownership\" that should be reviewed by a lawyer. details: is there any clause providing for joint or shared ownership of intellectual property between the parties to the contract?": 
"Show the sections of this contract (if there are any) related to the \"Joint IP Ownership\". Is there any clause providing for joint or shared ownership of intellectual property between the parties to the contract?",

    "highlight the parts (if any) of this contract related to \"license grant\" that should be reviewed by a lawyer. details: does the contract contain a license granted by one party to its counterparty?": 
"Show the sections of this contract (if there are any) related to the \"License Grant\". Is there a clause granting a license by one party to its counterparty?",

    "highlight the parts (if any) of this contract related to \"non-transferable license\" that should be reviewed by a lawyer. details: does the contract limit the ability of a party to transfer the license being granted to a third party?": 
"Show the sections of this contract (if there are any) related to the \"Non-Transferable License\". Is the contract limited to the ability of a party to transfer the license being granted to a third party?",

    "highlight the parts (if any) of this contract related to \"affiliate license-licensor\" that should be reviewed by a lawyer. details: does the contract contain a license grant by affiliates of the licensor or that includes intellectual property of affiliates of the licensor?\u00a0": 
"Show the sections of this contract (if there are any) related to the \"Affiliate License-Licensor\". Is the contract containing a license grant by affiliates of the licensor or that includes intellectual property of affiliates of the licensor?",

    "highlight the parts (if any) of this contract related to \"affiliate license-licensee\" that should be reviewed by a lawyer. details: does the contract contain a license grant to a licensee (incl. sublicensor) and the affiliates of such licensee/sublicensor?": 
"Show the sections of this contract (if there are any) related to the \"Affiliate License-Licensee\". Is the contract containing a license grant to a licensee (incl. sublicensor) and the affiliates of such licensee/sublicensor?",

    "highlight the parts (if any) of this contract related to \"unlimited/all-you-can-eat-license\" that should be reviewed by a lawyer. details: is there a clause granting one party an \u201centerprise,\u201d \u201call you can eat\u201d or unlimited usage license?": 
"Show the sections of this contract (if there are any) related to the \"Unlimited/All-You-Can-Eat License\". Is there a clause granting one party an \u201centerprise,\u201d \u201call you can eat\u201d or unlimited usage license?",

    "highlight the parts (if any) of this contract related to \"irrevocable or perpetual license\" that should be reviewed by a lawyer. details: does the contract contain a\u00a0 license grant that is irrevocable or perpetual?": 
"Show the sections of this contract (if there are any) related to the \"Irrevocable or Perpetual License\". Is the contract containing a license grant that is perpetual or irrevocable?",

    "highlight the parts (if any) of this contract related to \"source code escrow\" that should be reviewed by a lawyer. details: is one party required to deposit its source code into escrow with a third party, which can be released to the counterparty upon the occurrence of certain events (bankruptcy,\u00a0 insolvency, etc.)?": 
"Show the sections of this contract (if there are any) related to the \"Source Code Escrow\". Is one party required to deposit its source code into escrow with a third party, which can be released to the counterparty upon the occurrence of certain events (bankruptcy, insolvency, etc.)?",

    "highlight the parts (if any) of this contract related to \"post-termination services\" that should be reviewed by a lawyer. details: is a party subject to obligations after the termination or expiration of a contract, including any post-termination transition, payment, transfer of ip, wind-down, last-buy, or similar commitments?": 
"Show the sections of this contract (if there are any) related to the \"Post-Termination Services\". Is a party subject to obligations after the expiration or termination of a contract, including any post-termination transfer of ip, transition, payment, wind-down, last-buy, or similar commitments?",

    "highlight the parts (if any) of this contract related to \"audit rights\" that should be reviewed by a lawyer. details: does a party have the right to\u00a0 audit the books, records, or physical locations of the counterparty to ensure compliance with the contract?": 
"Show the sections of this contract (if there are any) related to the \"Audit Rights\". Is there a clause related to a party having the right to audit the records, books, or physical locations of the counterparty to ensure compliance with the contract?",

    "highlight the parts (if any) of this contract related to \"uncapped liability\" that should be reviewed by a lawyer. details: is a party\u2019s liability uncapped upon the breach of its obligation in the contract? this also includes uncap liability for a particular type of breach such as ip infringement or breach of confidentiality obligation.": 
"Show the sections of this contract (if there are any) related to the \"Uncapped Liability\". Is there a clause in the contract related to a party's uncapped liability upon the breach of its obligation in the contract? Including uncap liability for a particular type of breach such as breach of confidentiality obligation or IP infringement.",

    "highlight the parts (if any) of this contract related to \"cap on liability\" that should be reviewed by a lawyer. details: does the contract include a cap on liability upon the breach of a party\u2019s obligation? this includes time limitation for the counterparty to bring claims or maximum amount for recovery.": 
"Show the sections of this contract (if there are any) related to the \"Cap on Liability\". Is there a clause in the contract related to a cap on liability upon the breach of a party's obligation? Including time limitation for the counterparty to bring claims or maximum amount for recovery.",

    "highlight the parts (if any) of this contract related to \"liquidated damages\" that should be reviewed by a lawyer. details: does the contract contain a clause that would award either party liquidated damages for breach or a fee upon the termination of a contract (termination fee)?": 
"Show the sections of this contract (if there are any) related to the \"Liquidated Damages\". Is there a clause in the contract that would award either party liquidated damages for breach or a fee upon the termination of a contract?",

    "highlight the parts (if any) of this contract related to \"warranty duration\" that should be reviewed by a lawyer. details: what is the duration of any\u00a0 warranty against defects or errors in technology, products, or services\u00a0 provided under the contract?": 
"Show the sections of this contract (if there are any) related to the \"Warranty Duration\". This is the duration of any warranty against defects or errors in products, technology, or services provided under the contract?",

    "highlight the parts (if any) of this contract related to \"insurance\" that should be reviewed by a lawyer. details: is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?": 
"Show the sections of this contract (if there are any) related to the \"Insurance\". Are there any requirement for insurance that must be maintained by one party for the benefit of the counterparty?",

    "highlight the parts (if any) of this contract related to \"covenant not to sue\" that should be reviewed by a lawyer. details: is a party restricted from contesting the validity of the counterparty\u2019s ownership of intellectual property or otherwise bringing a claim against the counterparty for matters unrelated to the contract?": 
"Show the sections of this contract (if there are any) related to the \"Covenant Not to Sue\". Are there any restrictions on a party from contesting the validity of the counterparty's ownership of intellectual property or otherwise bringing a claim against the counterparty for matters unrelated to the contract?",

    "highlight the parts (if any) of this contract related to \"third party beneficiary\" that should be reviewed by a lawyer. details: is there a non-contracting party who is a beneficiary to some or all of the clauses in the contract and therefore can enforce its rights against a contracting party?": 
"Show the sections of this contract (if there are any) related to the \"Third Party Beneficiary\". Are there any non-contracting parties who is a beneficiary to some or all of the clauses in the contract and therefore can enforce its rights against a contracting party?",
}
# Create new test set
data_different_wording = change_question('../data/test.json', q_map_different_wording)
dump_json('../data/test_little_different_wording.json', data_different_wording)

In [30]:
# Calculate the jaccard distance between the two tokenized questions
jaccard_data = []
for k,v in q_map_different_wording.items():
    l1 = tk.encode(k)
    l2 = tk.encode(v)
    jaccard_data.append((k,v,jaccard_distance(l1,l2)))

    
np.mean([x[2] for x in jaccard_data])

0.47926506903515276

In [31]:
pd.DataFrame([(x, x.split('"')[1].split('"')[0]) for x in q_map_different_wording.values()]).to_clipboard()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


**4. Natural wording**

This a complete rephrasing in a natural sense by a senior law student

In [32]:
#Manual mapping
q_map_very_different_wording = {
    "highlight the parts (if any) of this contract related to \"document name\" that should be reviewed by a lawyer. details: the name of the contract": 
"What is the name of this legal document?",

    "highlight the parts (if any) of this contract related to \"parties\" that should be reviewed by a lawyer. details: the two or more parties who signed the contract": 
"Who are the parties to this legal document? The parties to the contract are the two or more parties who signed the contract.",

    "highlight the parts (if any) of this contract related to \"agreement date\" that should be reviewed by a lawyer. details: the date of the contract": 
"What is the agreement date of this contract? Agreement Date means the date as of which this Agreement is dated.",

    "highlight the parts (if any) of this contract related to \"effective date\" that should be reviewed by a lawyer. details: the date when the contract is effective\xa0": 
"In contract law, the effective date is the date that an agreement or transaction between or among signatories becomes binding. What is the effective date of this contract?",

    "highlight the parts (if any) of this contract related to \"expiration date\" that should be reviewed by a lawyer. details: on what date will the contract's initial term expire?": 
"What is the expiration date of the initial terms of this contract? The expiration of a contract puts an end to all the engagements of the parties, except to those which arise from the non- fulfillment of obligations created during its existence",

    "highlight the parts (if any) of this contract related to \"renewal term\" that should be reviewed by a lawyer. details: what is the renewal term after the initial term expires? this includes automatic extensions and unilateral extensions with prior notice.": 
"What sections are related to the \"Renewal Term\" in this contract? Renewal is keeping an existing arrangement in force for an additional period of time, such as a lease, a promissory note, insurance policy or any other contract. Renewal usually requires a writing or some action which evidences the new term.",

    "highlight the parts (if any) of this contract related to \"notice period to terminate renewal\" that should be reviewed by a lawyer. details: what is the notice period required to terminate renewal?": 
"Show everything related to the notice required to terminate renewal",

    "highlight the parts (if any) of this contract related to \"governing law\" that should be reviewed by a lawyer. details: which state/country's law governs the interpretation of the contract?": 
"What is the \"Governing Law\" of this contract?. Governing law is the law stipulated in a contract to determine a dispute",

    "highlight the parts (if any) of this contract related to \"most favored nation\" that should be reviewed by a lawyer. details: is there a clause that if a third party gets better terms on the licensing or sale of technology/goods/services described in the contract, the buyer of such technology/goods/services under the contract shall be entitled to those better terms?": 
"Is there any sections of this contract related to most favored nation?. Most Favored Nations (MFN) clauses (also known as antidiscrimination clauses or most-favored customer clauses) are common in business today. These provisions require that the supplier will treat a particular customer no worse than all other customers.",

    "highlight the parts (if any) of this contract related to \"non-compete\" that should be reviewed by a lawyer. details: is there a restriction on the ability of a party to compete with the counterparty or operate in a certain geography or business or technology sector?\u00a0": 
"Is there a any sections of this contract related to \"Non-Compete\". A non-compete agreement is a contract between two parties, usually two individuals or one company and one individual, in which one of the individuals promises not to compete with the other individual or company once their relationship with the company has ended. That is, he or she will not start, join, or buy a business that is similar to, and in competition with, the other.",

    "highlight the parts (if any) of this contract related to \"exclusivity\" that should be reviewed by a lawyer. details: is there an exclusive dealing\u00a0 commitment with the counterparty? this includes a commitment to procure all \u201crequirements\u201d from one party of certain technology, goods, or services or a prohibition on licensing or selling technology, goods or services to third parties, or a prohibition on\u00a0 collaborating or working with other parties), whether during the contract or\u00a0 after the contract ends (or both).": 
"Highlight any parts of this contract related to an \"Exclusivity\" clause. An exclusivity clause is part of a bigger legal document that restricts the signer from buying, selling, or promoting any goods or services from any person or company other than the issuing company associated with the contract. In other words, the company or individual works exclusively with the issuer of the contract.",

    "highlight the parts (if any) of this contract related to \"no-solicit of customers\" that should be reviewed by a lawyer. details: is a party restricted from contracting or soliciting customers or partners of the counterparty, whether during the contract or after the contract ends (or both)?": 
"Show everything related to \"No-Solicit of Employees\". This is a clause that prohibits one party from soliciting the customers of another party during a specified period of time",

    "drop highlight the parts (if any) of this contract related to \"competitive restriction exception\" that should be reviewed by a lawyer. details: this category includes the exceptions or carveouts to non-compete, exclusivity and no-solicit of customers above.": 
"--------------------------------Show the sections of this contract (if there are any) related to the \"Competitive Restriction Exception\". This is an exception or carveouts to the non-compete, exclusivity, and no-solicit of customers clauses",

    "highlight the parts (if any) of this contract related to \"no-solicit of employees\" that should be reviewed by a lawyer. details: is there a restriction on a party\u2019s soliciting or hiring employees and/or contractors from the\u00a0 counterparty, whether during the contract or after the contract ends (or both)?": 
"Show everything related to \"No-Solicit of Employees\". This is a clause that prohibits one party from soliciting the employees of another party during a specified period of time",

    "highlight the parts (if any) of this contract related to \"non-disparagement\" that should be reviewed by a lawyer. details: is there a requirement on a party not to disparage the counterparty?": 
"Show everything related to \"Non-Disparagement\" in this contract?. Non-disparagement clauses prevent parties from making derogatory comments about the other party.",

    "highlight the parts (if any) of this contract related to \"termination for convenience\" that should be reviewed by a lawyer. details: can a party terminate this\u00a0 contract without cause (solely by giving a notice and allowing a waiting\u00a0 period to expire)?": 
"Is there a clause related to \"Termination for Convenience\"?. A contractual right to terminate an agreement for any reason. It may also be referred to as termination without cause. A right to terminate for convenience usually requires the terminating party to provide a certain period of notice before the termination is effective and usually in writing",

    "drop highlight the parts (if any) of this contract related to \"rofr/rofo/rofn\" that should be reviewed by a lawyer. details: is there a clause granting one party a right of first refusal, right of first offer or right of first negotiation to purchase, license, market, or distribute equity interest, technology, assets, products or services?": 
"----------------Show any clauses related to \"Right of First Refusal, Right of First Offer, Right of First Negotiation\". This is a clause granting one party a right of first refusal, right of first offer or right of first negotiation to purchase, license, market, or distribute equity interest, technology, assets, products or services",

    "highlight the parts (if any) of this contract related to \"change of control\" that should be reviewed by a lawyer. details: does one party have the right to terminate or is consent or notice required of the counterparty if such party undergoes a change of control, such as a merger, stock sale, transfer of all or substantially all of its assets or business, or assignment by operation of law?": 
"Highlight any parts of this contract related to the \"Change of Control\". A provision in an agreement giving a party certain rights (such as consent, payment or termination) in connection with a change in ownership or management of the other party to the agreement.",

    "highlight the parts (if any) of this contract related to \"anti-assignment\" that should be reviewed by a lawyer. details: is consent or notice required of a party if the contract is assigned to a third party?": 
"Show any section of this contract related to \"Anti-Assignment\". An anti-assignment clause prevents either of the parties to a contract from assigning tasks to a third party without the consent of the non-assigning party.",

    "highlight the parts (if any) of this contract related to \"revenue/profit sharing\" that should be reviewed by a lawyer. details: is one party required to share revenue or profit with the counterparty for any technology, goods, or\u00a0services?": 
"Highlight all sections related to \"Revenue/Profit Sharing\". Revenue sharing is a somewhat flexible concept that involves sharing operating profits or losses among associated financial actors. Revenue sharing can exist as a profit-sharing system that ensures each entity is compensated for its efforts.",

    "drop highlight the parts (if any) of this contract related to \"price restrictions\" that should be reviewed by a lawyer. details: is there a restriction on the\u00a0 ability of a party to raise or reduce prices of technology, goods, or\u00a0 services provided?": 
"-------Show the sections of this contract (if there are any) related to the \"Price Restrictions\". This is a restriction on the ability of a party to raise or reduce prices of technology, goods, or services provided",

    "highlight the parts (if any) of this contract related to \"minimum commitment\" that should be reviewed by a lawyer. details: is there a minimum order size or minimum amount or units per-time period that one party must buy from the counterparty under the contract?": 
"Highlight any parts related to \"Minimum Commitment\". Clause sets out the minimum quantities of the goods or products that the buyer must purchase under the agreement and could include remedies for the buyer's failure to meet the minimum purchase requirement",

    "highlight the parts (if any) of this contract related to \"volume restriction\" that should be reviewed by a lawyer. details: is there a fee increase or consent requirement, etc. if one party\u2019s use of the product/services exceeds certain threshold?": 
"Highlight any section related to \"Volume Restriction\" in this contract. This is a fee consent or increase requirement, etc. if one party's use of the product/services exceeds certain threshold",

    "highlight the parts (if any) of this contract related to \"ip ownership assignment\" that should be reviewed by a lawyer. details: does intellectual property created\u00a0 by one party become the property of the counterparty, either per the terms of the contract or upon the occurrence of certain events?": 
"Highlight the sections of this contract related to the \"IP Ownership Assignment\". The assignment of intellectual property (IP) refers to the process by which ownership of work product created for one party is assigned to another party?",

    "highlight the parts (if any) of this contract related to \"joint ip ownership\" that should be reviewed by a lawyer. details: is there any clause providing for joint or shared ownership of intellectual property between the parties to the contract?": 
"Show the sections of this contract (if there are any) related to the \"Joint IP Ownership\". Joint ownership of intellectual property refers to the sharing of intellectual property (IP) rights to a particular invention between two or more parties. It usually occurs as a result of two or more people co-inventing a patentable product, creative work, design, or concept",

    "highlight the parts (if any) of this contract related to \"license grant\" that should be reviewed by a lawyer. details: does the contract contain a license granted by one party to its counterparty?": 
"Show the sections of this contract related to the \"License Grant\". The object of the a license grant clause is to grant permission to the one party to use certain intellectual property rights of the other party",

    "highlight the parts (if any) of this contract related to \"non-transferable license\" that should be reviewed by a lawyer. details: does the contract limit the ability of a party to transfer the license being granted to a third party?": 
"Show the sections of this contract related to the \"Non-Transferable License\". Clauses ensuring that the entire agreement, or some of its rights, obligations, and terms; may not be transferred to a party that has not signed the original contract?",
    
    "highlight the parts (if any) of this contract related to \"affiliate license-licensor\" that should be reviewed by a lawyer. details: does the contract contain a license grant by affiliates of the licensor or that includes intellectual property of affiliates of the licensor?\u00a0": 
"Show the sections of this contract (if there are any) related to the \"Affiliate License-Licensor\". A licensing agreement is a contract between a licensor and licensee in which the licensee gains access to the licensor's intellectual property. Does this legal document include a license grant that includes IP of partners of the licensor?",

    "highlight the parts (if any) of this contract related to \"affiliate license-licensee\" that should be reviewed by a lawyer. details: does the contract contain a license grant to a licensee (incl. sublicensor) and the affiliates of such licensee/sublicensor?": 
"Show the sections of this contract (if there are any) related to the \"Affiliate License-Licensee\". A licensing agreement is a contract between a licensor and licensee in which the licensee gains access to the licensor's intellectual property. Is the legal document containing a license grant to a licensee (incl. sublicensor) and the affiliates of such licensee/sublicensor?",

    "highlight the parts (if any) of this contract related to \"unlimited/all-you-can-eat-license\" that should be reviewed by a lawyer. details: is there a clause granting one party an \u201centerprise,\u201d \u201call you can eat\u201d or unlimited usage license?": 
"Show the sections of this contract related to the \"Unlimited/All-You-Can-Eat License\". Are there any clauses giving one of the contracting parties and unlimited usage license also refeered to as an all you can eat license?",

    "highlight the parts (if any) of this contract related to \"irrevocable or perpetual license\" that should be reviewed by a lawyer. details: does the contract contain a\u00a0 license grant that is irrevocable or perpetual?": 
"Show the sections of this contract related to the \"Irrevocable or Perpetual License\". The terms “irrevocable” and “perpetual” mean different things. A perpetual license is one that has no given end. An irrevocable license cannot be cut short. Either a perpetual license or a license with a stated term can be either revocable or irrevocable.",

    "highlight the parts (if any) of this contract related to \"source code escrow\" that should be reviewed by a lawyer. details: is one party required to deposit its source code into escrow with a third party, which can be released to the counterparty upon the occurrence of certain events (bankruptcy,\u00a0 insolvency, etc.)?": 
"Show the sections of this contract related to the \"Source Code Escrow\". Source code escrow is the deposit of the source code of software with a third-party escrow agent. Escrow is typically requested by a party licensing software, to ensure maintenance of the software instead of abandonment or reelease in occurrence of special events.?",

    "drop highlight the parts (if any) of this contract related to \"post-termination services\" that should be reviewed by a lawyer. details: is a party subject to obligations after the termination or expiration of a contract, including any post-termination transition, payment, transfer of ip, wind-down, last-buy, or similar commitments?": 
"----Show the sections of this contract related to the \"Post-Termination Services\". Is there a contractual obligation of one party after termination or expiration of a contract, including but not limited to transfer of ip, transition, payment, wind-down, last-buy, or similar commitments?",

    "highlight the parts (if any) of this contract related to \"audit rights\" that should be reviewed by a lawyer. details: does a party have the right to\u00a0 audit the books, records, or physical locations of the counterparty to ensure compliance with the contract?": 
"Show the sections of this contract related to the \"Audit Rights\". This is clause that entitles one party to review another party including but not limited to work, records, physical locations, and books which may include self-assessments, third-party audits and other, official documents detailing the sufficiency of internal systems and controls?",

    "highlight the parts (if any) of this contract related to \"uncapped liability\" that should be reviewed by a lawyer. details: is a party\u2019s liability uncapped upon the breach of its obligation in the contract? this also includes uncap liability for a particular type of breach such as ip infringement or breach of confidentiality obligation.": 
"Show the sections of this contract related to the \"Uncapped Liability\". A clause stating that there is no limit to the damage a party incurs if things go wrong, and thus no limit to the money to be paid out in respect of that damage upon the breach of the contract. Includes but not limited to ip infringement and breach of confidentiality obligations.",

    "drop highlight the parts (if any) of this contract related to \"cap on liability\" that should be reviewed by a lawyer. details: does the contract include a cap on liability upon the breach of a party\u2019s obligation? this includes time limitation for the counterparty to bring claims or maximum amount for recovery.": 
"---Show the sections of this contract (if there are any) related to the \"Cap on Liability\". Is there a clause in the contract related to a limitation or cap on the liability of the party upon the breach of a party's obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.",

    "highlight the parts (if any) of this contract related to \"liquidated damages\" that should be reviewed by a lawyer. details: does the contract contain a clause that would award either party liquidated damages for breach or a fee upon the termination of a contract (termination fee)?": 
"Show the sections of this contract (if there are any) related to the \"Liquidated Damages\". A fixed or determined sum agreed by the parties to a contract to be payable on breach of the contract by one of the parties",

    "drop highlight the parts (if any) of this contract related to \"warranty duration\" that should be reviewed by a lawyer. details: what is the duration of any\u00a0 warranty against defects or errors in technology, products, or services\u00a0 provided under the contract?": 
"----Show the sections of this contract (if there are any) related to the \"Warranty Duration\". This is the duration of any warranty against defects or errors in products, technology, or services provided under the contract?",

    "drop highlight the parts (if any) of this contract related to \"insurance\" that should be reviewed by a lawyer. details: is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?": 
"----Show the sections of this contract (if there are any) related to the \"Insurance\". Are there any requirement for insurance that must be maintained by one party for the benefit of the counterparty?",

    "highlight the parts (if any) of this contract related to \"covenant not to sue\" that should be reviewed by a lawyer. details: is a party restricted from contesting the validity of the counterparty\u2019s ownership of intellectual property or otherwise bringing a claim against the counterparty for matters unrelated to the contract?": 
"Show the sections of this contract related to the \"Covenant Not to Sue\". A covenant not to sue is a legal agreement that obliges a party that could seek damages to refrain from suing the party that it has cause against? This could be for reasons of intellectual property, or for other reasons.",

    "highlight the parts (if any) of this contract related to \"third party beneficiary\" that should be reviewed by a lawyer. details: is there a non-contracting party who is a beneficiary to some or all of the clauses in the contract and therefore can enforce its rights against a contracting party?": 
"Show the sections of this contract related to the \"Third Party Beneficiary\". A third-party beneficiary is a person or business that benefits from the terms of a contract made between two other parties. Are there any such parties in the contract that can enforce its rights against a contracting party?",
}
# Create new test set
data_vert_different_wording, unique_q = change_question('../data/test_full.json', q_map_very_different_wording, drop_question_not_found=True, return_unique_q=True)
dump_json('../data/test_very_different_wording.json', data_vert_different_wording)

Dropped 714 questions not found in question_map out of 4182 questions


In [35]:
k

'data'

In [36]:
# Calculate the jaccard distance between the two tokenized questions
jaccard_data = []
for k,v in q_map_very_different_wording.items():
    l1 = tk.encode(k)
    l2 = tk.encode(v)
    jaccard_data.append((k,v,jaccard_distance(l1,l2)))

    
np.mean([x[2] for x in jaccard_data])

0.6852908895710341

In [7]:
len(unique_q)

34

In [65]:
pd.DataFrame([(v, x.split('"')[1].split('"')[0]) for x,v in unique_q.items()]).to_clipboard(index=False)